In [1]:
import json
import sys
import os
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize.stanford import StanfordTokenizer
from nltk.parse.corenlp import CoreNLPParser
from nltk.corpus import stopwords
from collections import defaultdict
from gensim import corpora
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
import gensim
import re
import string
import nltk
import numpy as np
import pandas as pd
#nltk.download() #use if nltk cries for any language-specific docs

In [2]:
"""ay=str()
for file in os.listdir('Models\word2vec\English CoNLL17 corpus - gensim skipgram - 10'):
    full_filename = "%s/%s" % ('Models\word2vec\English CoNLL17 corpus - gensim skipgram - 10', file)
    if full_filename.endswith('.txt'):
        with open(full_filename,'r', encoding="ISO-8859-1") as fi:
            ay=fi.readlines()
"""
subs=[]
for filename in os.listdir('Subs'):
    if filename.endswith(".json"):
        with open(os.path.join('Subs', filename)) as json_file:
            json_text = json.load(json_file)
            print(filename)
            subs.append(json_text)

OUTPUT200acres.json
OUTPUTBraincels.json
OUTPUTClownworldwar.json
OUTPUTCringeAnarchy.json
OUTPUTDarkEnlightenment.json
OUTPUTdeep_ecology.json
OUTPUTfrenworld.json
OUTPUTGenderCritical.json
OUTPUTHBD.json
OUTPUTmetacanada.json
OUTPUTMGTOW.json
OUTPUTShitPoliticsSays.json
OUTPUTTheNewRight.json
OUTPUTThe_Donald.json
OUTPUTThe_Donald2.json
OUTPUTThe_Donald3.json


In [3]:
#not used
def sub2vocab(sub):
    documents=sub2docs(sub)
    temp=[]
    stop = set(stopwords.words('english'))
    #for doc in documents:
        #print(word_tokenize(doc))
    texts =[
        [word for word in document.lower().split() if word not in stop]
        for document in documents
    ]
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    texts = [
        [token for token in text if frequency[token] > 1]
        for text in texts]
    return texts

In [11]:
#not used even more
"""def sub2vocab(sub):
    raw=str()
    for i, j in sub.items():
        raw+=i
        raw+=' '
        for o,p in j.items():
            raw+=p
            raw+=' '
        raw+='\n'
    temp=[]
    stop = set(stopwords.words('english'))
    for i in word_tokenize(raw.lower()):
        if i not in stop:
            temp.append(i)
    frequency = defaultdict(int)
    for word in temp:
        frequency[word] += 1
    temp = set([token for token in temp if frequency[token] > 1])
    return temp"""

"def sub2vocab(sub):\n    raw=str()\n    for i, j in sub.items():\n        raw+=i\n        raw+=' '\n        for o,p in j.items():\n            raw+=p\n            raw+=' '\n        raw+='\n'\n    temp=[]\n    stop = set(stopwords.words('english'))\n    for i in word_tokenize(raw.lower()):\n        if i not in stop:\n            temp.append(i)\n    frequency = defaultdict(int)\n    for word in temp:\n        frequency[word] += 1\n    temp = set([token for token in temp if frequency[token] > 1])\n    return temp"

In [4]:
def sub2docs(sub):
    documents=[]
    trans_table = str.maketrans("", "", string.punctuation+"’”")
    for i, j in sub.items():
        temp=re.sub(r'\d+','',i.lower().translate(trans_table)).replace('\n', '')
        documents.append(temp)
        for o,p in j.items():
            temp=re.sub(r'\d+', '',p.lower().translate(trans_table)).replace('\n', '')
            documents.append(temp)
    frequency = defaultdict(int)
    for document in documents:
        for token in document.split():
            frequency[token] += 1
    documents = [
        [token for token in document.split() if frequency[token] > 1]
        for document in documents]
    return documents

In [7]:
"""def removePunctuation(words):
    trans_table = str.maketrans("", "", string.punctuation)
    return set([s.translate(trans_table) for s in words])"""

'def removePunctuation(words):\n    trans_table = str.maketrans("", "", string.punctuation)\n    return set([s.translate(trans_table) for s in words])'

In [19]:
def target_wv2distances(wv,missing):
    dists=np.zeros((len(wv.vocab),len(wv.vocab)))
    words=[]
    for word in wv.vocab.keys():
        words.append(word)
    for i in range(len(words)):
        if words[i] not in missing:
            if i%500 == 0:
                print(i,len(words))
            for j in range(len(words)):
                if words[j] not in missing:
                    j+=i
                    if j<len(words):
                        if words[j] not in missing:
                            dists[i,j]=wv.n_similarity([words[i]],[words[j]])
    return dists

In [18]:
def ref_wv2distances(target_wv,reference_wv):
    dists=np.zeros((len(target_wv.vocab),len(target_wv.vocab)))
    words=[]
    for word in target_wv.vocab.keys():
        words.append(word)
    start=0
    missing=[]
    for i in range(len(words)):
        if i%500 == 0:
            print(i,len(words))
        for j in range(len(words)):
            j+=start
            try:
                if j<len(words):
                    dists[i,j]=reference_wv.n_similarity([words[i]],[words[j]])
            except Exception as e:
                word=re.findall('''"word '(.+?)' not in vocabulary''', str(e))[0]
                if word not in missing:
                    #pass
                    print("type error: " + str(e))
                    missing.append(word)
                else:
                    pass
        start+=1
    return dists, missing


In [7]:
def docs2wv(docs,name='model'):
    model = Word2Vec(docs, size=300, window=5, workers=3, sg=1, min_count=1, negative=8)
    if name!=None:
        model.save(name+".model")
    word_vectors = model.wv
    frequency = defaultdict(int)
    for document in docs:
        for token in document:
            frequency[token] += 1
    return word_vectors, frequency

In [8]:
def remove_diagonal(dists):
    for i in range(dists.shape[0]):
        for j in range(dists.shape[1]):
            if i==j:
                dists[i,j]=0
    return dists

In [9]:
docs=[]
for sub in subs:
    doc=sub2docs(sub)
    docs.extend(doc)
docs[0]

['welcome', 'announcement']

In [10]:
target_wv,frequency=docs2wv(docs,name='target')
ref_wv=gensim.models.KeyedVectors.load_word2vec_format(os.path.join(os.path.dirname('Models\word2vec\English_CoNLL17\ '), 'model.bin'), binary=True).wv

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [20]:
dists_ref,missing = ref_wv2distances(target_wv,ref_wv)
dists_t = target_wv2distances(target_wv,missing)
dists_ref = remove_diagonal(dists_ref)
dists_t = remove_diagonal(dists_t)

0 28188
type error: "word 'chadright' not in vocabulary"
type error: "word 'shoahd' not in vocabulary"
type error: "word 'thuleanperspective' not in vocabulary"
type error: "word 'plebbit' not in vocabulary"
type error: "word 'memelords' not in vocabulary"
type error: "word 'riotim' not in vocabulary"
type error: "word 'pinesim' not in vocabulary"
type error: "word 'lolberts' not in vocabulary"
type error: "word 'racres' not in vocabulary"
type error: "word 'fashwave' not in vocabulary"
type error: "word 'antiwomen' not in vocabulary"
type error: "word 'altright' not in vocabulary"
type error: "word 'judeochristians' not in vocabulary"
type error: "word 'igtt' not in vocabulary"
type error: "word '“the' not in vocabulary"
type error: "word 'rstrangestatistics' not in vocabulary"
type error: "word 'rrakers' not in vocabulary"
type error: "word 'elsagate' not in vocabulary"
type error: "word 'selfdestructive' not in vocabulary"
type error: "word 'racemixing' not in vocabulary"
type error

type error: "word 'chadfish' not in vocabulary"
type error: "word 'mgtows' not in vocabulary"
type error: "word 'stblackopscel' not in vocabulary"
type error: "word 'ragefuel' not in vocabulary"
type error: "word 'rshort' not in vocabulary"
type error: "word 'globolistcel' not in vocabulary"
type error: "word 'jbwpill' not in vocabulary"
type error: "word 'magratm' not in vocabulary"
type error: "word 'incelswithouthate' not in vocabulary"
type error: "word 'soyfag' not in vocabulary"
type error: "word 'inkels' not in vocabulary"
type error: "word 'tallcels' not in vocabulary"
type error: "word 'norwooding' not in vocabulary"
type error: "word 'gymcels' not in vocabulary"
type error: "word 'soyim' not in vocabulary"
type error: "word '😎😎😎😎' not in vocabulary"
type error: "word 'mladies' not in vocabulary"
type error: "word 'mstow' not in vocabulary"
type error: "word 'gigachads' not in vocabulary"
type error: "word '“haha' not in vocabulary"
type error: "word 'edgelord' not in vocabula

type error: "word 'causationcorrelation' not in vocabulary"
type error: "word 'yeeted' not in vocabulary"
type error: "word 'tradcon' not in vocabulary"
type error: "word 'fryheit' not in vocabulary"
type error: "word 'vilvanathan' not in vocabulary"
type error: "word '“whiteness' not in vocabulary"
type error: "word 'mrbeast' not in vocabulary"
type error: "word 'nonneoreactionaries' not in vocabulary"
type error: "word 'neoreactionarieshowever' not in vocabulary"
type error: "word 'rdebatede' not in vocabulary"
type error: "word 'subredditmessagecomposetordarkenlightenment' not in vocabulary"
type error: "word 'httpswwwyoutubecomwatchvzmnjymsmifeatureyoutube' not in vocabulary"
type error: "word 'neocommunism' not in vocabulary"
type error: "word '“right' not in vocabulary"
type error: "word '“good' not in vocabulary"
type error: "word '“human' not in vocabulary"
type error: "word 'bluehaired' not in vocabulary"
type error: "word '“real' not in vocabulary"
type error: "word 'rmgtow' 

type error: "word 'ractuallesbians' not in vocabulary"
type error: "word '“girldick' not in vocabulary"
type error: "word '“just' not in vocabulary"
type error: "word '“everyone' not in vocabulary"
type error: "word 'girldick' not in vocabulary"
type error: "word 'thereive' not in vocabulary"
type error: "word '“girl' not in vocabulary"
type error: "word '“as' not in vocabulary"
type error: "word 'xlinked' not in vocabulary"
type error: "word 'womanas' not in vocabulary"
type error: "word 'contrapoints' not in vocabulary"
type error: "word '“trans' not in vocabulary"
type error: "word '“feminist' not in vocabulary"
type error: "word '“look' not in vocabulary"
type error: "word '“skinny' not in vocabulary"
type error: "word '“female' not in vocabulary"
type error: "word 'transbased' not in vocabulary"
type error: "word 'romanticsexual' not in vocabulary"
type error: "word 'repeaked' not in vocabulary"
type error: "word 'crosssex' not in vocabulary"
type error: "word 'heshethey' not in v

type error: "word 'namalt' not in vocabulary"
type error: "word 'antimisogyny' not in vocabulary"
type error: "word 'noncis' not in vocabulary"
type error: "word '“of' not in vocabulary"
type error: "word '“pass' not in vocabulary"
type error: "word 'womanso' not in vocabulary"
type error: "word '“male' not in vocabulary"
type error: "word '“they' not in vocabulary"
type error: "word 'peoplesexual' not in vocabulary"
type error: "word 'familyfriends' not in vocabulary"
type error: "word 'laughablei' not in vocabulary"
type error: "word '“gender' not in vocabulary"
type error: "word 'antiautogynephilia' not in vocabulary"
type error: "word 'rthisneverhappens' not in vocabulary"
type error: "word 'thoughyoure' not in vocabulary"
type error: "word '“and' not in vocabulary"
type error: "word 'themits' not in vocabulary"
type error: "word 'itcant' not in vocabulary"
type error: "word 'rgcdebatesqt' not in vocabulary"
type error: "word 'detrans' not in vocabulary"
type error: "word '“like' n

...

type error: "word 'antiqueer' not in vocabulary"
type error: "word 'nonfemale' not in vocabulary"
type error: "word '“enough' not in vocabulary"
type error: "word '“who' not in vocabulary"
type error: "word '“boys' not in vocabulary"
type error: "word 'httpswwwabcnetaureligiontransgenderismthelatestantifeministwedgeofthelefthttpswwwabcnetaureligiontransgenderismthelatestantifeministwedgeoftheleft' not in vocabulary"
type error: "word '“free' not in vocabulary"
type error: "word 'caitlyns' not in vocabulary"
type error: "word 'ocasiocortez' not in vocabulary"
type error: "word 'secondwave' not in vocabulary"
type error: "word '•any' not in vocabulary"
type error: "word '“did' not in vocabulary"
type error: "word 'murdersuicide' not in vocabulary"
type error: "word 'cismales' not in vocabulary"
type error: "word '“hot' not in vocabulary"
type error: "word 'sexismmisogyny' not in vocabulary"
type error: "word 'fantasysci' not in vocabulary"
type error: "word 'gendercynical' not in vocabul

type error: "word 'virtuesignalling' not in vocabulary"
type error: "word 'whitesupremacist' not in vocabulary"
type error: "word 'rcanada' not in vocabulary"
type error: "word 'albaghdadi' not in vocabulary"
type error: "word 'metacanada' not in vocabulary"
type error: "word 'snclavalin' not in vocabulary"
type error: "word 'liberallite' not in vocabulary"
type error: "word '“conservative' not in vocabulary"
type error: "word 'rwsjw' not in vocabulary"
type error: "word 'sockboys' not in vocabulary"
type error: "word 'feesmaybe' not in vocabulary"
type error: "word 'ogft' not in vocabulary"
type error: "word '“student' not in vocabulary"
type error: "word 'wilsonraybould' not in vocabulary"
type error: "word 'rtoronto' not in vocabulary"
type error: "word 'trudope' not in vocabulary"
type error: "word 'equityantiwhite' not in vocabulary"
type error: "word 'turdeau' not in vocabulary"
type error: "word 'utdb' not in vocabulary"
type error: "word 'harvo' not in vocabulary"
type error: "

type error: "word 'rconservative' not in vocabulary"
type error: "word 'powertripping' not in vocabulary"
type error: "word 'archive™' not in vocabulary"
type error: "word 'raskreddit' not in vocabulary"
type error: "word 'spezwe' not in vocabulary"
type error: "word '“brown' not in vocabulary"
type error: "word 'dunningkruger' not in vocabulary"
type error: "word '“kids' not in vocabulary"
type error: "word 'genderstereotyped' not in vocabulary"
type error: "word 'protrump' not in vocabulary"
type error: "word 'beforewhen' not in vocabulary"
type error: "word 'narcissismtrump' not in vocabulary"
type error: "word 'fascismi' not in vocabulary"
type error: "word 'rshitliberalssay' not in vocabulary"
type error: "word 'tidepods' not in vocabulary"
type error: "word '“republicans' not in vocabulary"
type error: "word '“unless' not in vocabulary"
type error: "word 'rmeira' not in vocabulary"
type error: "word 'reeeeeeeeeeee' not in vocabulary"
type error: "word 'rworldpolitics' not in voca

type error: "word 'magaing' not in vocabulary"
type error: "word '“former' not in vocabulary"
type error: "word 'kêk' not in vocabulary"
type error: "word 'gaetzs' not in vocabulary"
type error: "word '“chicken' not in vocabulary"
type error: "word 'hamberder' not in vocabulary"
type error: "word 'strozk' not in vocabulary"
type error: "word '“spying' not in vocabulary"
type error: "word '“deep' not in vocabulary"
type error: "word '“deny' not in vocabulary"
type error: "word 'whitesplain' not in vocabulary"
type error: "word 'pantifa' not in vocabulary"
type error: "word '“climate' not in vocabulary"
type error: "word 'trainofthought' not in vocabulary"
type error: "word 'soyvage' not in vocabulary"
type error: "word 'rtard' not in vocabulary"
type error: "word '“snowflake' not in vocabulary"
type error: "word 'illhan' not in vocabulary"
type error: "word 'freedomloving' not in vocabulary"
type error: "word 'podestas' not in vocabulary"
type error: "word 'donktum' not in vocabulary"
t

500 28188
1000 28188
1500 28188
2000 28188
2500 28188
3000 28188
3500 28188
4000 28188
4500 28188
5000 28188
5500 28188
6000 28188
6500 28188
7000 28188
7500 28188
8000 28188
8500 28188
9000 28188
9500 28188
10000 28188
10500 28188
11000 28188
11500 28188
12000 28188
12500 28188
13000 28188
13500 28188
14000 28188
14500 28188
15000 28188
15500 28188
16000 28188
16500 28188
17000 28188
17500 28188
18000 28188
18500 28188
19000 28188
19500 28188
20000 28188
20500 28188
21000 28188
21500 28188
22000 28188
22500 28188
23000 28188
23500 28188
24000 28188
24500 28188
25000 28188
25500 28188
26000 28188
26500 28188
27000 28188
27500 28188
28000 28188
0 28188
500 28188
1000 28188
1500 28188
2000 28188
2500 28188
3000 28188
3500 28188
4000 28188
4500 28188
5500 28188
6000 28188
6500 28188
7000 28188
7500 28188
8000 28188
8500 28188
9000 28188
9500 28188
10500 28188
11000 28188
11500 28188
12000 28188
12500 28188
13000 28188
13500 28188
14500 28188
15000 28188
15500 28188
16000 28188
16500 28188

In [16]:
"""e='''type error: "word 'memelords' not in vocabulary"'''
word=re.findall(''' error: "word '(.+?)' not in vocabulary''', str(e))[0]
print(word)"""

'e=\'\'\'type error: "word \'memelords\' not in vocabulary"\'\'\'\nword=re.findall(\'\'\' error: "word \'(.+?)\' not in vocabulary\'\'\', str(e))[0]\nprint(word)'

In [21]:
ranks = pd.DataFrame(columns=['Value','Count','Prominence'])
dists_comp = np.absolute(dists_t-dists_ref)

In [22]:
i=0
#dists_comp=np.absolute(dists_t-dists_ref)
for word in target_wv.vocab.keys():
    if i%500 == 0:
        print(i)
    temp=np.sum(dists_comp[i,:])+np.sum(dists_comp[:,i])
    ranks.loc[word]=[temp,frequency[word],0]
    i+=1

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000


In [23]:
ranks['Value']=ranks['Value']/ranks['Value'].mean()

In [24]:
ranks['Prominence']=ranks['Value']*np.log(ranks['Count'])

In [25]:
ranks=ranks.sort_values('Value',ascending=False)

In [32]:
ranks.to_csv('result.csv')

In [26]:
filtered=ranks[ranks['Count']>10]

In [31]:
pd.options.display.max_rows = 150
filtered[:120]

,Value,Count,Prominence
og,1.524391,11.0,3.655330
ne,1.496264,11.0,3.587885
😭,1.466942,14.0,3.871345
inb,1.461684,20.0,4.378815
finnish,1.446687,12.0,3.594882
itt,1.445096,11.0,3.465190
😂😂,1.441345,11.0,3.456193
gaia,1.427232,11.0,3.422352
lul,1.425211,16.0,3.951525
rt,1.412809,11.0,3.387768
